In [ ]:
import numpy as np
import cupy as cp
from cupyx.profiler import benchmark

# CuPy Basic Examples

## Example 1: Creating Arrays

In [ ]:
# Creates an array of size (100x1000x1000) on the CPU using NumPy
def create_array_CPU():
    x_cpu = np.ones((100,1000,1000))
    return x_cpu

# Creates an array of size (100x1000x1000) on the GPU using CuPy
def create_array_GPU():
    x_gpu = cp.ones((100,1000,1000))

    # Ensures code finishes executing on GPU before moving to next line
    cp.cuda.Stream.null.synchronize()
    return x_gpu

In [ ]:
x_cpu = create_array_CPU()
print(type(x_cpu))

In [ ]:
x_gpu = create_array_GPU()
print(type(x_gpu))

In [ ]:
%timeit create_array_CPU()
%timeit create_array_GPU()

## Example 2: Data Transfer

### Moving arrays to device

In [ ]:
x_cpu = create_array_CPU()
print(type(x_cpu))
print(x_cpu[0])

In [ ]:
x_gpu = cp.asarray(x_cpu) 
print(type(x_gpu))
x_gpu *=2
print(x_gpu[0])

### Moving arrays from device back to host

In [ ]:
x_cpu = cp.asnumpy(x_gpu)  # move the array to the host.
print(x_cpu[0])
print(type(x_cpu))

or

In [ ]:
x_cpu = x_gpu.get()  # move the array to the host.
print(x_cpu[0])
print(type(x_cpu))

## Example 3: Matrix Operations

In [ ]:
# Create a large random matrix
size = 1000
matrix = np.random.rand(size, size)

# CPU version
def cpu_matrix_ops(matrix):
    for i in range(size):
        for j in range(size):
            # Perform some computation on each element of the matrix
            matrix[i][j] = np.sin(matrix[i][j]) + np.cos(matrix[i][j])


# CPU version without loop
def cpu_matrix_ops_noloop(matrix):
    matrix = np.sin(matrix) + np.cos(matrix)

# GPU version
def gpu_matrix_ops(matrix):

    # Convert matrix to a CuPy array
    matrix = cp.asarray(matrix)

    # Perform matrix operations in parallel on the GPU
    matrix = cp.sin(matrix) + cp.cos(matrix)

    # Convert matrix back to a NumPy array
    #matrix = cp.asnumpy(matrix)

# Compare performance of CPU and GPU versions
%timeit cpu_matrix_ops(matrix)              # CPU version
%timeit cpu_matrix_ops_noloop(matrix)         # CPU version
%timeit gpu_matrix_ops(matrix)                # GPU version

In [ ]:
# GPU unoptimized version
def gpu_matrix_ops_unoptimized(matrix):

    matrix = cp.asarray(matrix)

    for i in range(size):
        for j in range(size):
            # Perform some computation on each element of the matrix
            matrix[i][j] = cp.sin(matrix[i][j]) + cp.cos(matrix[i][j])

In [ ]:
cpu_matrix_ops(matrix)

In [ ]:
gpu_matrix_ops_unoptimized(matrix)

### Adding data transfer to the mix

In [ ]:
# GPU version
def gpu_matrix_ops(matrix):

    # Convert matrix to a CuPy array
    matrix = cp.asarray(matrix)

    # Perform matrix operations in parallel on the GPU
    matrix = cp.sin(matrix) + cp.cos(matrix)

    # Convert matrix back to a NumPy array
    matrix = cp.asnumpy(matrix)

In [ ]:
%timeit gpu_matrix_ops(matrix) 

# Gaia Data Examples

In [ ]:
import cupy as cp
import numpy as np
DeviceContext = cp.cuda.Device
dtype = cp.float32
from numba import config
config.DISABLE_JIT = True


import pandas as pd
import sys
sys.path.append("../gaia_tools/")
import transformation_constants
import transformation_functions

In [ ]:
print('Grabbing needed columns')
trans_needed_columns = ['source_id', 'ra', 'dec', 'r_est', 'pmra', 'pmdec', 'radial_velocity']

print('Importing DR3')
path = '/home/svenpoder/DATA/Gaia_DR3/Poder_vc_DR3_input.csv'
gaia_dr3 = pd.read_csv(path)
icrs_data = gaia_dr3[trans_needed_columns]

In [ ]:
print(icrs_data.shape)
print(icrs_data.columns)

In [ ]:
# Declaring parameters related to the Sun's position and velocity

v_sun = transformation_constants.V_SUN

r_0 = 8277.0
z_0 = 25.0

v_sun[0][0] = 11.1
v_sun[1][0] = 251.5*(r_0/8277)
v_sun[2][0] = 8.59*(r_0/8277)

In [ ]:
def get_galactocentric_data_CPU(icrs_data, z_0, r_0, v_sun):

    galcen_data = transformation_functions.get_transformed_data(icrs_data,
                                        include_cylindrical = True,
                                        z_0 = z_0,
                                        r_0 = r_0,
                                        v_sun = v_sun,
                                        is_bayes = True,
                                        NUMPY_LIB = np,
                                        dtype = dtype)
    return galcen_data

In [ ]:
%timeit get_galactocentric_data_CPU(icrs_data, z_0, r_0, v_sun)

## Coordinate transformation on GPU

In [ ]:
# Transfer the data from host to device!
icrs_data_gpu = cp.asarray(icrs_data, dtype=dtype)

In [ ]:
def get_galactocentric_data_GPU(icrs_data, z_0, r_0, v_sun):

    galcen_data = transformation_functions.get_transformed_data(icrs_data,
                                            include_cylindrical = True,
                                            z_0 = z_0,
                                            r_0 = r_0,
                                            v_sun = v_sun,
                                            is_bayes = True,
                                            NUMPY_LIB = cp,
                                            dtype = dtype)
    return galcen_data

In [ ]:
%timeit get_galactocentric_data_GPU(icrs_data_gpu, z_0, r_0, v_sun)

## Covariance propagation on CPU

In [ ]:
import covariance_generation as cov

In [ ]:
C_icrs = cov.generate_covmat(gaia_dr3)

In [ ]:
C_icrs.shape

In [ ]:
galcen_data_cpu = get_galactocentric_data_CPU(icrs_data, z_0, r_0, v_sun)

In [ ]:
def get_galactocentric_cov_matrix_CPU(C_icrs, icrs_data, galcen_data, z_0, r_0):


    # ["ra", "dec","r_est","pmra","pmdec","radial_velocity"] -> [:,1::]
    galactocentric_cov = cov.transform_cov_matrix(C = C_icrs, 
                                        df = icrs_data.to_numpy()[:,1::],
                                        coordinate_system = 'Cartesian',
                                        z_0 = z_0,
                                        r_0 = r_0,
                                        is_bayes = True,
                                        NUMPY_LIB = np,
                                        dtype = dtype)

    # ["x", "y","r","phi","v_r","v_phi"] -> [0,1,6,7,8,9]
    cyl_cov_gpu = cov.transform_cov_matrix(C = galactocentric_cov, 
                                        df = galcen_data[:,[0,1,6,7,8,9]],
                                        coordinate_system = 'Cylindrical',
                                        z_0 = z_0,
                                        r_0 = r_0,
                                        is_bayes = False,
                                        NUMPY_LIB = np,
                                        dtype = dtype)

In [ ]:
%timeit get_galactocentric_cov_matrix_CPU(C_icrs, icrs_data, galcen_data_cpu, z_0, r_0)

## Covariance Propagation on GPU

In [ ]:
C_icrs_gpu = cp.asarray(C_icrs, dtype=cp.float32)

In [ ]:
galcen_data_gpu = get_galactocentric_data_GPU(icrs_data_gpu, z_0, r_0, v_sun)

In [ ]:
def get_galactocentric_cov_matrix_GPU(C_icrs, icrs_data, galcen_data, z_0, r_0):


    # ["ra", "dec","r_est","pmra","pmdec","radial_velocity"] -> [:,1::]
    galactocentric_cov = cov.transform_cov_matrix(C = C_icrs, 
                                        df = icrs_data[:,1::],
                                        coordinate_system = 'Cartesian',
                                        z_0 = z_0,
                                        r_0 = r_0,
                                        is_bayes = True,
                                        NUMPY_LIB = cp,
                                        dtype = dtype)

    # ["x", "y","r","phi","v_r","v_phi"] -> [0,1,6,7,8,9]
    cyl_cov_gpu = cov.transform_cov_matrix(C = galactocentric_cov, 
                                        df = galcen_data[:,[0,1,6,7,8,9]],
                                        coordinate_system = 'Cylindrical',
                                        z_0 = z_0,
                                        r_0 = r_0,
                                        is_bayes = False,
                                        NUMPY_LIB = cp,
                                        dtype = dtype)

In [ ]:
%timeit get_galactocentric_cov_matrix_GPU(C_icrs_gpu, icrs_data_gpu, galcen_data_gpu, z_0, r_0)